<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Complaints Analysis using Vantage and AWS Bedrock
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233c'><b>Introduction:</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Analyzing consumer complaints using audio files conversations involves leveraging advanced technologies like large language models (LLM) and AWS Bedrock to extract insights from unstructured data. This process can be applied to various sectors, including financial services, telecommunications, and e-commerce, to improve customer experience and protect consumer rights.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233c'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Configuring the environment</li>
    <li>Configuring AWS CLI</li>
    <li>Define Anthropic model and Prompt</li>
</ol>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Configuring the environment</b>

In [1]:
%%capture
!pip install --upgrade -r requirements.txt --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>Please restart the kernel after executing these two lines. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Import the required libraries</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [2]:
import timeit
import boto3
import numpy as np
import pandas as pd
from tqdm import tqdm
import getpass

# teradataml
from teradataml import *

from IPython.display import Markdown, Audio
display.max_rows = 5
pd.set_option('display.max_colwidth', None)

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>2. Configuring AWS CLI</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following cell will prompt us for the following information:</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
<li><b>aws_access_key_id</b>: Enter your AWS access key ID</li>
<li><b>aws_secret_access_key</b>: Enter your AWS secret access key</li>
<li><b>region name</b>: Enter the AWS region you want to configure (e.g., us-east-1)</li>
<ol>

In [3]:
def configure_aws():
    print("configure the AWS CLI")
    # enter the access_key/secret_key
    access_key = getpass.getpass("aws_access_key_id ")
    secret_key = getpass.getpass("aws_secret_access_key ")
    region_name = getpass.getpass("region name")

    #set to the env
    !aws configure set aws_access_key_id {access_key}
    !aws configure set aws_secret_access_key {secret_key}
    !aws configure set default.region {region_name}

In [4]:
does_access_key_exists = !aws configure get aws_access_key_id

if len(does_access_key_exists) == 0:
    configure_aws()

In [5]:
!aws configure list

      Name                    Value             Type    Location
      ----                    -----             ----    --------
   profile                <not set>             None    None
access_key     ****************GXKN shared-credentials-file    
secret_key     ****************u8mf shared-credentials-file    
    region                us-east-1      config-file    ~/.aws/config


<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Define Anthropic model and Prompt</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following section defines the type of Claude 3 model used. Here we use <b>Claude 3 Sonnet</b></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here the architecture diagram of audio analysis using Bedrock.</p>
<center><img src="images/architecture_bedrock_audio.png" alt="architecture"  width=515 height=526/></center>

<br/>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The project architecture consists of three main steps:</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>We initiate the process by uploading an audio file to the source folder of our S3 bucket, which we've configured to send an event notification to a Lambda function whenever a new object is created.</li>
    <li>We receive the event notification in our Lambda function, <code>s3_trigger_transcribe</code> which then initiates an Amazon Transcribe job using the uploaded file as the source media. The results of the transcription are saved to the transcription folder of the same S3 bucket.</li>
    <li>We leverage an Event Rule from Amazon EventBridge to monitor Amazon Transcribe jobs that start with "summarizer-" and have either a COMPLETED or FAILED status. When we detect a job in one of these states, we automatically send the Transcribe job details to our Lambda function, eventbridge-bedrock-inference. This function takes the transcript, formats it, and crafts an instruction prompt for a Bedrock large language model (LLM) to distill the essence of the audio content. Finally, we store the summarized results in the processed folder of the same S3 bucket.</li>
</ol>

In [6]:
import boto3

bucket_name = "csae-usecases"
import os
import glob
import time

s3 = boto3.client("s3")

def upload_files_to_s3(mp3_files):
    print('\n Step 1. Uploading files.')
    for audio_file_name in mp3_files:
        file_path = os.path.join(os.getcwd(), "audio_files", audio_file_name)

        # Upload the file to the S3 bucket
        object_name = "consumer_complaints/speech_analysis/source/" + audio_file_name
        with open(file_path, "rb") as file:
            s3.upload_fileobj(file, bucket_name, object_name)
            print(f"File '{file_path}' uploaded to '{bucket_name}/{object_name}'")


def retrieve_summary(active_job_dict):
    print('\n Step 3. Generating the final output, please wait..')
    wait_time = 12 * len(active_job_dict)
    print('wait time: ', wait_time)
    time.sleep(wait_time)
    print('please wait...')
    prefix = "consumer_complaints/speech_analysis/processed/"

    # Call the list_objects_v2 method with the Prefix parameter
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    summaries_dict = {}
    for file_name, _job_name in active_job_dict.items():
        for obj in response.get("Contents", []):
            if _job_name in obj["Key"]:
                print(f"file_name: {file_name} | _job_name: {_job_name}")
                try:
                    # Get the object from S3
                    obj = s3.get_object(
                        Bucket=bucket_name, Key=f"{prefix}{_job_name}.txt"
                    )

                    # Read the contents of the file and add to summary dict
                    summaries_dict[file_name] = obj["Body"].read().decode("utf-8")
                except Exception as e:
                    print(f"An error occurred: {e}")
    print('processing done..')
    return summaries_dict


def list_mp3_files(directory):
    mp3_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".mp3"):
                mp3_files.append(file)
    return mp3_files


def start_process():
    response = s3.list_buckets()
    directory_path = "./audio_files"
    mp3_files = list_mp3_files(directory_path)

    # Upload files to s3
    upload_files_to_s3(mp3_files)
    time.sleep(5)

    # Monitor transcription status
    # Create a Transcribe client
    transcribe = boto3.client("transcribe")
    active_job_names = []
    active_job_dict = {}
    # List active transcription jobs
    try:
        response = transcribe.list_transcription_jobs(Status="IN_PROGRESS")
        active_jobs = response["TranscriptionJobSummaries"]

        # Sort active jobs by creation time
        active_jobs.sort(key=lambda job: job["CreationTime"], reverse=True)

        # Print the list of active jobs
        if len(active_jobs) > 0:
            print("\n Step 2. Generating the transcription. \nActive transcription jobs:")
            for job in active_jobs:
                _job_name = job["TranscriptionJobName"]
                print(f"- {_job_name} ({job['TranscriptionJobStatus']})")

                job_name_response = transcribe.get_transcription_job(
                    TranscriptionJobName=_job_name
                )
                audio_file_name = job_name_response["TranscriptionJob"]["Media"][
                    "MediaFileUri"
                ].split("/")[-1]
                active_job_dict[audio_file_name] = _job_name

            # Assign the latest job name to job_name
            job_name = active_jobs[0]["TranscriptionJobName"]
            active_job_names.append(job_name)
            print(f"\n*** active_job_dict *** \n\n {active_job_dict}")

            print(f"\nThe latest transcription job is: {job_name}\n")
        else:
            print("No active transcription jobs found.")

    except transcribe.exceptions.BadRequestException as e:
        print(f"Error: {e}")
    except transcribe.exceptions.InternalFailureException as e:
        print(f"Error: {e}")
    except transcribe.exceptions.LimitExceededException as e:
        print(f"Error: {e}")

    max_retries = 180  # Maximum number of retries
    retry_delay = 15  # Delay between retries (in seconds)

    # Monitor/poll for transcription status
    retries = 0
    while retries < max_retries:
        try:
            response = transcribe.get_transcription_job(TranscriptionJobName=job_name)
            job_status = response["TranscriptionJob"]["TranscriptionJobStatus"]
            print(f"Job status: {job_status}")

            if job_status == "COMPLETED":
                transcription_file_uri = response["TranscriptionJob"]["Transcript"][
                    "TranscriptFileUri"
                ]
                print(f"Transcription file: {transcription_file_uri}")
                break
            elif job_status == "FAILED":
                failure_reason = response["TranscriptionJob"]["FailureReason"]
                print(f"Job failed: {failure_reason}")
                break
            else:
                print(f"Job is still in progress. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                retries += 1

        except transcribe.exceptions.BadRequestException as e:
            print(f"Error: {e}")
        except transcribe.exceptions.InternalFailureException as e:
            print(f"Error: {e}")
        except transcribe.exceptions.LimitExceededException as e:
            print(f"Error: {e}")

    # Retrieve the summary
    return retrieve_summary(active_job_dict)

In [7]:
summaries = start_process()


 Step 1. Uploading files.
File '/home/jovyan/JupyterLabRoot/UseCases/Complaints_Analysis_GenAI_Bedrock/audio_files/Conversation_3.mp3' uploaded to 'csae-usecases/consumer_complaints/speech_analysis/source/Conversation_3.mp3'
File '/home/jovyan/JupyterLabRoot/UseCases/Complaints_Analysis_GenAI_Bedrock/audio_files/audio1.mp3' uploaded to 'csae-usecases/consumer_complaints/speech_analysis/source/audio1.mp3'
File '/home/jovyan/JupyterLabRoot/UseCases/Complaints_Analysis_GenAI_Bedrock/audio_files/Conversation_2.mp3' uploaded to 'csae-usecases/consumer_complaints/speech_analysis/source/Conversation_2.mp3'
File '/home/jovyan/JupyterLabRoot/UseCases/Complaints_Analysis_GenAI_Bedrock/audio_files/Conversation_4.mp3' uploaded to 'csae-usecases/consumer_complaints/speech_analysis/source/Conversation_4.mp3'
File '/home/jovyan/JupyterLabRoot/UseCases/Complaints_Analysis_GenAI_Bedrock/audio_files/Conversation_1.mp3' uploaded to 'csae-usecases/consumer_complaints/speech_analysis/source/Conversation_1

In [8]:
from IPython.display import display, Markdown

for file_name, response in summaries.items():
    desc = f'''<hr style="height:1px;border:none;background-color:#00233C;">
        <p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>For file: {file_name} </b></p>'''
    display(Markdown(desc))
    display(Audio((f"audio_files/{file_name}")))
    display(Markdown(response))

<hr style="height:1px;border:none;background-color:#00233C;">
        <p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>For file: audio1.mp3 </b></p>

#### Complaint:
spk_0: I applied for a home equity loan through Discover Home loans and was denied due to the fact there. appraisal found my home to be wakened. My home is occupied and can be easily proven. They did not allow to be to dispute this forever sent me a copy of the appraisal until after they had closed my loan. They have also disregarded all my documentation showing my residency and have not provided any. information on why my house was labelled weakened.

#### Classification: 
- Complaint
- **Reasoning with Chain-of-Thought**: The speaker expresses dissatisfaction with Discover Home Loans regarding the denial of a home equity loan application, lack of transparency in the appraisal process, and the company's disregard for evidence of residency. These elements indicate that the transcript is a complaint.

#### Topic:
- Mortgage Application
- **Reasoning with Chain-of-Thought**: The central issue revolves around the speaker's application for a home equity loan, which falls under the Mortgage Application topic.

#### Sentiment:
- Negative
- **Reasoning with Chain-of-Thought**: The speaker uses words like "denied," "disregarded," and expresses frustration with the lack of information provided by the company, indicating a negative sentiment towards the situation.

#### Summary:
- The speaker is dissatisfied with Discover Home Loans for denying their home equity loan application based on an appraisal labeling their occupied home as "weakened," without providing the appraisal report or explanation for the decision, despite the speaker's evidence of residency.
- **Reasoning with Chain-of-Thought**: The summary captures the key elements of the complaint: the speaker's loan application was denied due to an appraisal issue, the company did not provide the appraisal report or explanation for the decision, and the speaker's documentation of residency was disregarded.

<hr style="height:1px;border:none;background-color:#00233C;">
        <p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>For file: Conversation_1.mp3 </b></p>

#### Complaint:
- [Provide the actual complaint from the audio file here.]
spk_0: Hi, I'm John. What's the issue with your credit card payment? Emily, I tried to make a payment online but it says my card is declined. I know I have enough money in my account.

#### Classification:
- Complaint
- **Reasoning with Chain-of-Thought**: The transcript contains a statement expressing dissatisfaction or grievance from the customer (Emily) about their credit card payment being declined despite having sufficient funds in their account. This qualifies as a complaint.

#### Topic:
- Payment Trouble
- **Reasoning with Chain-of-Thought**: The central issue or subject of the complaint is related to the customer facing difficulties or troubles with making a payment using their credit card. This falls under the "Payment Trouble" topic.

#### Sentiment:
- Negative
- **Reasoning with Chain-of-Thought**: The customer's tone and language convey frustration and dissatisfaction with the payment issue, indicating a negative sentiment.

#### Summary:
- A customer complained about their credit card payment being declined despite having enough funds in their account.
- **Reasoning with Chain-of-Thought**: The key elements of the transcript are the customer (Emily) trying to make an online payment, the payment being declined despite sufficient funds, and the customer expressing frustration or dissatisfaction with this issue. These elements form the essence of the summary.

<hr style="height:1px;border:none;background-color:#00233C;">
        <p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>For file: Conversation_4.mp3 </b></p>

#### Complaint:
- I'm trying to transfer money to my savings account but it's not going through. I need the money urgently

#### Classification:
- Complaint
- **Reasoning with Chain-of-Thought**: The transcript contains a statement expressing dissatisfaction with the inability to transfer money to a savings account, which is a grievance or issue being raised. Therefore, it can be classified as a complaint.

#### Topic:
- Payment Trouble
- **Reasoning with Chain-of-Thought**: The main issue or subject of the complaint revolves around having trouble making a payment or transfer of funds, specifically from a checking account to a savings account. This falls under the "Payment Trouble" topic category.

#### Sentiment:
- Negative
- **Reasoning with Chain-of-Thought**: The sentiment expressed in the complaint is negative, as the speaker is dissatisfied with the inability to transfer money and mentions needing the money urgently, indicating frustration or negativity towards the situation.

#### Summary:
- The customer was unable to transfer money from their checking account to their savings account due to a temporary system issue, causing frustration as they urgently needed the funds.
- **Reasoning with Chain-of-Thought**: The summary captures the main points of the complaint, which are the customer's inability to transfer money between accounts, the urgency of needing the funds, and the underlying cause being a temporary system issue on the bank's end.

<hr style="height:1px;border:none;background-color:#00233C;">
        <p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>For file: Conversation_2.mp3 </b></p>

#### Complaint:
- [This is not a complaint. It's a conversation between a customer support representative and a user trying to reset their password.]

#### Classification:
- Non-Complaint
- **Reasoning with Chain-of-Thought**: The transcript does not contain any statements expressing dissatisfaction or grievance. It is simply a conversation where the user is seeking assistance from customer support to reset their password due to a technical issue. There are no elements of a complaint present.

#### Topic:
- Account/Login Issue
- **Reasoning with Chain-of-Thought**: The central issue in the conversation revolves around the user's inability to reset their password due to an invalid email address issue. This falls under the broad category of account or login-related problems.

#### Sentiment:
- Neutral
- **Reasoning with Chain-of-Thought**: The overall tone of the conversation is neutral, without any strong positive or negative emotions expressed. The user is simply seeking assistance, and the customer support representative is trying to resolve the issue in a professional manner.

#### Summary:
- A user contacted customer support to reset their password but encountered an issue with their email address being marked as invalid, which the support representative was able to resolve.
- **Reasoning with Chain-of-Thought**: The summary captures the key elements of the conversation, which involved the user's inability to reset their password due to an email address issue, and the customer support representative's successful resolution of the problem.

<hr style="height:1px;border:none;background-color:#00233C;">
        <p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>For file: Conversation_3.mp3 </b></p>

#### Complaint:
- [Provide the actual complaint from the audio file here.]
spk_0: Three call support engineer. Hi, I'm John. What's the issue with your mortgage application? Sarah? I submitted my application a week ago but I haven't heard back. I'm getting worried that something is wrong.

#### Classification: 
- Complaint
- **Reasoning with Chain-of-Thought**: The transcript contains a statement expressing dissatisfaction or grievance regarding the delay in processing a mortgage application, which fits the definition of a complaint.

#### Topic:
- Mortgage Application
- **Reasoning with Chain-of-Thought**: The central issue or subject of the complaint revolves around the delay in processing and getting a response for a submitted mortgage application.

#### Sentiment:
- Negative
- **Reasoning with Chain-of-Thought**: The sentiment expressed in the transcript is negative, as the speaker (Sarah) is worried and dissatisfied due to the delay in receiving a response for the mortgage application.

#### Summary:
- The speaker is frustrated about not receiving a response for her mortgage application submitted a week ago.  
- **Reasoning with Chain-of-Thought**: The key elements of the transcript are the speaker's concern and dissatisfaction over the delay in processing and getting a response for their mortgage application, which has been submitted a week prior.

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024. All Rights Reserved
        </div>
    </div>
</footer>